<a id='start'></a>

# Digitizing protocol for Bitty's *Mycena* spatial analysis

This is a record of our point pattern analysis of Bitty's *Mycena* plots from the West Eugene Wetlands. 

A sample photo looks like this:

<img src='P3_06_sample.jpg', width=350,height=350>

I used [GIMP](www.gimp.org) to correct perspective, trim borders, and convert to .png format (don't know why, just prefer this to jpeg), resulting in:

<img src="P3_06.png", width=350,height=350>

This was done for all 18 plots. We can feed these into **R**:

In [2]:
## raster graphic reader of choice:
library('png')
library('sp')
library('repr')

## set our working directory:
setwd('/home/daniel/Documents/Bitty_meadow_analysis/analysis')

## assign a variable name for the path to our photo directory:
mdir <- '/home/daniel/Documents/Bitty_meadow_analysis/photos_mod/'

## get all of our file names into one object, still in R environment
## but using system() command
photolist <- system('ls /home/daniel/Documents/Bitty_meadow_analysis/photos_mod', intern=TRUE)

## read our photo into memory. We start with #6 for kicks, this is P3_06.png
img <- readPNG(paste(mdir,photolist[6], sep=''))

We can then use **R**'s basic plot and raster imaging functions to show this photo, and assign axes in centimeters, from 0 to 100 cm, to match our actual 1-meter-square plots. 

In [3]:
options(repr.plot.width = 5, repr.plot.height = 5) ## using repr to control size of R plotter outputs

In [5]:
plot(1, type = 'n', xlim = c(0,100), ylim = c(0,100), xlab = '', ylab = '')
rasterImage(img, 0,0,100,100)

<img src=P3_06_sampleplot.png>

We need to digitize the photos for further analysis. We are interested in the spatial relationship between the *Mycena* mushrooms and the bunchgrasses. The mushroom events can probably be considered simple points, as they seem discrete and separate enough to delineate as points. We can examine them later for second-order-type relationships (clustering). The bunch grasses, however, are variable and large enough on this scale that we should probably digitize them as polygons. Each photo will have multiple polygons, each polygon representing one bunch of grass. 

For this, I will use the [sp package](https://cran.r-project.org/web/packages/sp/index.html), to create a spatial polygons object for each photo. I coded a small function to facilitate the process. Very primitive, but it works. Still in **R**:

In [ ]:
digpols <- function() {
        require('sp')
        iter <- 1
        an <- 'y'
        grasspols <- NULL
        allpols <- list()
        while (an != 'n') {
                grass <- locator()
                grasspols <- Polygon(rbind(cbind(grass$x, grass$y), cbind(grass$x[1],grass$y[1])))
                bb <- Polygons(list(grasspols), ID = paste('grass',iter,sep=''))
                allpols[[iter]] <- bb
                plot(SpatialPolygons(allpols), lwd=2, add=TRUE)
                an <- readline(prompt = "Another(y/n)? ")
                if (an == 'y') { iter <- iter + 1 } else {break}}
        cc <- SpatialPolygons(allpols)
return(cc)
}

As long as the photo of the plot we are interested in is the current graphics object on display in R's plotter, we can use this function to digitize our bunch grasses:

In [ ]:
p3_06_grass <- digpols() 

Using this I can draw polygons around the grasses. To aid in this, while drawing on the R plotter I also keep an image viewer open with the image that has good zoom capabilites (I use EOG on ubuntu). Here's an example:

In [2]:
plot(1, type = 'n', xlim = c(0,100), ylim = c(0,100), xlab = '', ylab = '')
rasterImage(img, 0,0,100,100)
p3_06_grass <- digpols() ## our homemade function above.

## check our work:
plot(1, type = 'n', xlim = c(0,100), ylim = c(0,100), xlab = '', ylab = '')
rasterImage(img, 0,0,100,100)
plot(p3_06_grass, lwd = 3, add = TRUE)

![](p3_06_sample2.png)

For our mushroom points, let's assign points to every cluster of mushrooms. First bring up our image of the plot, fresh. 

In [ ]:
img <- readPNG(paste(mdir,photolist[6], sep=''))
plot(1, type = 'n', xlim = c(0,100), ylim = c(0,100), xlab = '', ylab = '')
rasterImage(img, 0,0,100,100)

Use locator() to mark the locations of the mushrooms in the picture. Really helped here to have a good image viewer on the side to really zoom in and hunt for the mushrooms. 

In [ ]:
mushpts_p3_06 <- locator()

Then make this into a spatial points object:

In [ ]:
sp.mushpts_p3_06 <-SpatialPoints(cbind(mushpts_p3_06$x, mushpts_p3_06$y))
plot(sp.mushpts_p3_06, pch = 17, cex = 2, lwd = 2, col='red', add=TRUE)

![](p3_06_mush.png)

Removing the photograph background, our digitized data look like this:

In [ ]:
plot(p3_06_grass, lwd = 3, add = FALSE)
plot(sp.mushpts_p3_06, pch = 17, cex = 2, lwd = 2, col='red', add=TRUE)

![](p3_06_sample_pols_mush.png)

Now we save these as R data objects, for further analysis.

In [ ]:
save(p3_06_grass, file = 'p3_06_grass.rda')
save(sp.mushpts_p3_06, file = 'sp.mushpts_p3_06.rda')

And that's the method for digitizing these photos! 